In [2]:
import pandas as pd
import re
import string
import unicodedata

In [4]:
df = pd.read_csv('./vihallu-train.csv',encoding='utf-8')
df

,id,context,prompt,response,label
0,9b1ea51d-d1ff-45ba-8cf1-6a91328e8600,"Vào những năm 1870, hai nhà điêu khắc Augustus...","Vào những năm 1960, nơi nào trở thành trung tâ...",Quảng trường Washington là trung tâm của thế h...,extrinsic
1,db7a89c6-2a6a-42af-beef-58e557ecc819,Cách mạng Tháng Mười đã biến một cuộc chiến tr...,Cách mạng Tháng Mười đã khởi xướng chủ nghĩa p...,Sai. Cách mạng Tháng Mười đã đánh bại chủ nghĩ...,no
2,10fca062-d343-4eca-8434-93c7a8aa5e0e,Vị trí địa lý nằm giữa phương Đông và Địa Trun...,Dựa trên việc Đế quốc Ottoman không bao giờ ki...,Nền ẩm thực của Thổ Nhĩ Kỳ được xây dựng từ gi...,extrinsic
3,ece8eb9e-d6bb-407a-a567-d9531861c603,"Hồ Quý Ly trước có tên là Lê Quý Ly (黎季犛), biể...",Đời cháu thứ 12 của Hồ Hưng Dật sống ở đâu?,"Đời cháu thứ 12 của Hồ Hưng Dật, là Hồ Liêm, s...",no
4,b613217f-df2a-491e-8326-25811a31eb09,"Năm 2007, Bồ Đào Nha có khoảng 332.137 người n...","Từ thập niên 1990, tại sao xuất hiện một vài l...",Mặc dù Bồ Đào Nha đã đóng cửa biên giới hoàn t...,intrinsic
...,...,...,...,...,...
6995,ca2cfa61-c9ba-4923-b79e-def4af189ef8,Qua những hành động lừa dối bá tước Derby tại ...,Chiiến dịch muà hè đã tác động đếnn Edward như...,Chiến dịch mùa hè giúp Edward học hỏi từ những...,no
6996,26edc592-2a30-4bb8-956b-7764b7725f41,Tồn tại một số tranh chấp văn hóa giữa Malaysi...,"Để giải quyết tranh chấp văn hóa, Malaysia đã ...","Malaysia không chỉ nhượng bộ hoàn toàn, mà còn...",intrinsic
6997,c998fafc-6108-44b3-9bb5-dcd814e74013,Các triết gia Pháp sản sinh một số tác phẩm ch...,Mỹ dã ap dụn ly thuyet nào trong Tinh thần pha...,Nguyên tắc phân chia quyền lực được Mỹ áp dụng...,extrinsic
6998,a3c61cb3-2585-4c6f-bbe4-19a14ca7b09a,"Từ ""Yehudi"" (số nhiều Yehudim) trong tiếng Heb...",Từ Jeew trong tiếng Anh dc bắt nguồn từ tờ nào?,"Từ ""Jew"" trong tiếng Anh không chỉ bắt nguồn t...",extrinsic


13303


In [ ]:
special_case = {'dđ': 'đ'}
not_in_vietnamese = ['w', 'f', 'j', 'z', 'W', 'F', 'J', 'Z']

INVALID = 'aaaaa'

def remove_diacritics(text: str) -> str:
    nfkd = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in nfkd if unicodedata.category(ch) != 'Mn')

def replace_word(sentence: str, old: str, new: str) -> str:
    words = sentence.split()
    replaced = [new if w == old else w for w in words]
    return " ".join(replaced)

def modify_word(w):
    """
    Sửa lỗi các từ như "dđ" và các ký tự lặp lại (ví dụ: "haaa" thành "ha").
    Hàm này chỉ xử lý phần chữ cái của từ.
    """
    if len(w) == 0:
        return w
    
    # Xử lý các ký tự lặp lại
    if w[0].islower():
        for c in string.ascii_lowercase:
            while c + c in w:
                w = w.replace(c + c, c)
        for c in not_in_vietnamese:
            w = w.replace(c, '')
    
    # Xử lý các trường hợp đặc biệt
    for old, new in special_case.items():
        if old in w:
            w = w.replace(old, new)
            
    return w

def fix_sentence(s):
    """
    Sửa lỗi từng từ trong một câu trong khi vẫn giữ lại dấu câu.
    """
    # Tìm tất cả các từ (chuỗi chữ cái) và áp dụng hàm modify_word cho chúng
    # Mọi thứ khác (dấu câu, khoảng trắng) sẽ được giữ nguyên
    return re.sub(r'[a-zA-ZÀ-ỹ]+', lambda m: modify_word(m.group(0)), s)

In [97]:
df = pd.read_csv('./vihallu-train.csv',encoding='utf-8')
df['prompt'] = df['prompt'].apply(fix_sentence)

In [ ]:
num = 12

for i, (c, p, r) in enumerate(zip(df['context'], df['prompt'], df['response'])):
    new_response = remove_diacritics(r)
    response_dicts = {}

    for w, new_w in zip(r.split(' '), new_response.split(' ')):
        if response_dicts.get(new_w) is None:
            response_dicts[new_w] = w
        elif response_dicts[new_w] != w:
            response_dicts.get(w) == INVALID

    new_prompt = remove_diacritics(p)
    # if i == num:
    #     print(r)
    #     print(new_response)

    #     print(new_prompt)
    
    for w, new_w in zip(p.split(' '), new_prompt.split(' ')):
        if response_dicts.get(new_w) is not None and response_dicts.get(new_w) != INVALID:

            new_prompt = replace_word(new_prompt, new_w, response_dicts.get(new_w))
            # if i == num:
            #     print("replace", new_w, "->", response_dicts.get(new_w))
        else:
            new_prompt = replace_word(new_prompt, new_w, w)
            # if i == num:
            #     print("not replace", new_w, "->", w)
            
    # if i == num:
    #     print(new_prompt)
    df.loc[i, 'prompt'] = new_prompt


Do bề mặt Sao Kim có nhiều dung nham nguội chảy tràn, tạo ra địa hình gồ ghề và không ổn định. Ngoài ra, điều kiện ánh sáng yếu dưới bầu trời đầy mây cũng làm giảm tầm nhìn, gây khó khăn cho việc di chuyển.
Do be mat Sao Kim co nhieu dung nham nguoi chay tran, tao ra đia hinh go ghe va khong on đinh. Ngoai ra, đieu kien anh sang yeu duoi bau troi đay may cung lam giam tam nhin, gay kho khan cho viec di chuyen.
Tai sao con nguoi di bo tren hanh tinh nay kho khan?
not replace Tai -> Tai
not replace sao -> sao
not replace con -> con
replace nguoi -> nguội
replace di -> di
not replace bo -> bô
not replace tren -> tren
not replace hanh -> hanh
not replace tinh -> tinh
not replace nay -> này
replace kho -> khó
not replace khan? -> khăn?
Tai sao con nguội di bô tren hanh tinh này khó khăn?


In [77]:
df.to_csv('phuoc.csv', index=False, encoding='utf-8')

In [96]:
df.head(20)

,id,context,prompt,response,label
0,9b1ea51d-d1ff-45ba-8cf1-6a91328e8600,"Vào những năm 1870, hai nhà điêu khắc Augustus...","Vào những năm 1960, nơi nào trở thành trung tâ...",Quảng trường Washington là trung tâm của thế h...,extrinsic
1,db7a89c6-2a6a-42af-beef-58e557ecc819,Cách mạng Tháng Mười đã biến một cuộc chiến tr...,Cách mạng Tháng Mười đã khởi xướng chủ nghĩa p...,Sai. Cách mạng Tháng Mười đã đánh bại chủ nghĩ...,no
2,10fca062-d343-4eca-8434-93c7a8aa5e0e,Vị trí địa lý nằm giữa phương Đông và Địa Trun...,Dựa trên việc Đế quốc Ottoman không bao giờ ki...,Nền ẩm thực của Thổ Nhĩ Kỳ được xây dựng từ gi...,extrinsic
3,ece8eb9e-d6bb-407a-a567-d9531861c603,"Hồ Quý Ly trước có tên là Lê Quý Ly (黎季犛), biể...",Đời cháu thứ 12 của Hồ Hưng Dật sống ở đâu?,"Đời cháu thứ 12 của Hồ Hưng Dật, là Hồ Liêm, s...",no
4,b613217f-df2a-491e-8326-25811a31eb09,"Năm 2007, Bồ Đào Nha có khoảng 332.137 người n...","Từ thập niên 1990, tại sao xuất hiện một vài l...",Mặc dù Bồ Đào Nha đã đóng cửa biên giới hoàn t...,intrinsic
5,4a3e2996-af6a-430b-a5f3-c920e84bae5b,Thủy triều cao và sự hiện hữu của một lực lượn...,Địa điểm tiến công của quân đoàn X trong Chiến...,Địa điểm tiến công của Quân đoàn X trong Chiến...,no
6,0be1d385-0b0f-425f-8875-6c562d47a83b,Hiến chương Pháp ngữ 1977 xác định tiếng Pháp ...,Mặc dù Hiến chương Pháp ngữ 1977 khẳng định ti...,Mặc dù Hiến chương Pháp ngữ 1977 xác định tiến...,intrinsic
7,3e6f0c19-4222-4010-885a-7eb6718c88e2,Birmingham duy trì là thành phố tỉnh lẻ giàu c...,"Vào thập niên 70, Birmingham đã trở thành trun...","Không, điều này không đúng. Trong thập niên 70...",no
8,a637f00d-4286-4e74-8291-0092d025872e,Các chiến thắng của quân đội Trung Quốc cho th...,Adversarial Question: Tại sao quân đội Trung Q...,Quân đội Trung Quốc giành lợi thế nhờ vào tinh...,no
9,0ed6ac08-34c0-48f9-b637-932b55409ac6,Nhật Bản còn có một mỹ danh trong cộng đồng ng...,Vì sao Nhật Bản còn có cái tên Phù Tang?,Nhật Bản được gọi là Phù Tang vì theo truyền t...,extrinsic


In [17]:
s = set()

prompt = df['prompt']
for p in prompt:
    #tach cac tu cua p ra roi dua vao set
    words = p.split(' ')
    for w in words:
        s.add(w)

print(len(s))

dicts = {}
for w in s:
    dicts[w] = 0

for p in prompt:
    words = p.split(' ')
    
    for w in words:
        if w in dicts:
            dicts[w] += 1

dicts = dict(sorted(dicts.items(), key=lambda item: item[1], reverse=True))

with open('vocab.csv', 'w', encoding='utf-8') as f:
    for w in dicts:
        f.write(f'{w},{dicts[w]}\n')

12887
